In [ ]:
## Airbyte PGVector RAG Demo

This tutorial demonstrates how to use data stored in Airbyte's PGVector destination to perform Retrieval-Augmented Generation (RAG). You should use this destination when you intend to use PGVector for LLM specific vector operations like RAG.

As a practical example, we'll build a Assistant—an AI chatbot capable of answering questions related to simpsons episoded using data from multiple Airbyte-related sources.

#### Prerequisites:
* Vector data stored in Postgres with Vector colums via PGVector destination. In our case we are using data from kaggle.
* Postgresql DB with PGVector enabled
* Open AI key


### a. Install dependencies and import secrets



In [ ]:
!pip install sqlalchemy openai rich psycopg2 python-dotenv langchain-openai

In [ ]:
import openai
import json
import rich
from langchain_openai import OpenAIEmbeddings
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from google.colab import userdata


OPENAI_API_KEY = userdata.get('openai_api_key')
HOST =  userdata.get("db_host")
USERNAME =  userdata.get("db_username")
PASSWORD =  userdata.get("db_password")
DATABASE = userdata.get("db_name")

### b. Initialize open AI client and DB Engine


In [ ]:
openai.api_key = OPENAI_API_KEY

url = URL.create(
    "postgresql",
    host=HOST,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
)

engine = create_engine(url)

### d. Explore data stored in Posgresql

We need a few methods to embed user questions and make searches in DB

- Helper to embed the user question so we can the search for it in the DB.
- Function to get the context from the database using a user question as input.
- One to get the response from the chat assistant that will use the context using the method from previous step.

In [ ]:
from sqlalchemy import text
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def get_embedding_from_open_ai(question):
    print(f"Embedding user's query: {question}")
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    embedding_response = embeddings.embed_query(question)
    return embedding_response

QUERY_TEMPLATE = """
SELECT document_content,
    metadata->'title' as episode_title,
    metadata->'script_line_number' as script_line_number,
    metadata->'name' as character_name,
    metadata->'spoken_words' as spoken_words
FROM episode_spoken_words
ORDER BY embedding <-> :question_vector
LIMIT 5
"""

def get_context(question) -> str:
    # Get the embedding from OpenAI
    question_vector = get_embedding_from_open_ai(question)

    # Format the question vector as a string in the format expected by PostgreSQL
    question_vector_str = '[' + ','.join(map(str, question_vector)) + ']'

    # Use the text() function for raw queries with SQLAlchemy
    query = text(QUERY_TEMPLATE)

    # Execute the query, passing the vector as a bind parameter
    with engine.connect() as connection:
        result = connection.execute(query, {'question_vector': question_vector_str})

        # Format and return the result
        return ("\n\n" + "-" * 8 + "\n\n").join(
            [
                f"Episode {row.episode_title} | Line number: {row.script_line_number} | "
                f"Spoken Words: {row.spoken_words} | Character: {row.character_name}"
                for row in result
            ]
        )


def get_response(question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a Simpsons expert talking about Simpsons episodes."},
            {"role": "user", "content": question},
            {"role": "assistant", "content": f"Use only this information to answer the question: {get_context(question)}. Do not search on the internet."}
        ]
    )
    return response.choices[0].message.content



### d. Make questions

Finally, let's put all together and get a response from our assistant using the Simpsons database.

In [ ]:
question = "Talking about food"
response = get_response(question)
rich.print(response)